In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DB_PATH = "../db/global_dev.db"
conn = sqlite3.connect(DB_PATH)


In [4]:
# 1. List all tables in the database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
table_list = tables['name'].tolist()
print("Available tables:")
for idx, tbl in enumerate(table_list):
    print(f"{idx}: {tbl}")


Available tables:
0: basic_drinking_water_access
1: employment_to_population_15plus
2: immunization_dpt
3: hiv_prevalence_total
4: gdp_current_usd
5: gov_education_expenditure_percent_gdp
6: poverty_headcount_215
7: life_expectancy_total
8: health_expenditure_per_capita
9: gdp_per_capita_usd
10: access_to_electricity
11: school_enrollment_secondary
12: labor_force_total
13: gni_per_capita_atlas
14: managed_sanitation_access
15: unemployment_total_percent


In [5]:
# 2. Load one table (set index below)
table_idx = 0  # <--- Change this to choose another indicator!
table = table_list[table_idx]
print(f"Loading table: {table}")

df = pd.read_sql(f"SELECT * FROM '{table}'", conn)
print(f"Shape: {df.shape}")
df.head()


Loading table: basic_drinking_water_access
Shape: (266, 27)


,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,94.488806,94.770491,95.052176,95.333861,95.615545,95.897230,...,97.869023,97.869023,97.869023,97.869023,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,41.801456,42.481960,43.186326,43.896852,44.611063,45.409889,...,51.898122,52.733864,53.585369,54.393056,55.210540,56.049837,56.858234,57.626951,58.377556,59.107461
2,Afghanistan,AFG,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,27.441856,27.473580,29.674863,31.875589,34.094431,36.331226,...,56.858142,59.598158,62.369660,65.172162,68.008872,70.879941,73.784643,76.722255,79.693078,82.175371
3,Africa Western and Central,AFW,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,50.673598,51.595428,52.647025,53.700626,54.762999,55.833555,...,64.433944,65.516834,66.601361,67.678297,68.751476,69.827426,70.886937,71.942470,73.037463,74.158152
4,Angola,AGO,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,41.144310,42.254676,43.376800,44.363873,45.351335,46.336018,...,52.757346,53.541508,54.316931,55.084273,55.842910,56.591749,56.883037,57.167738,57.446726,57.719561


In [6]:
# 3. Inspect columns, types, missing values, unique countries, and years
print("Columns:", df.columns.tolist())
print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

# Guess typical columns
def guess_col(df, substrings):
    for col in df.columns:
        if any(sub in col.lower() for sub in substrings):
            return col
    return None

country_col = guess_col(df, ["country"])
year_col = guess_col(df, ["year"])
value_col = None
for col in df.columns:
    if col not in [country_col, year_col] and pd.api.types.is_numeric_dtype(df[col]):
        value_col = col
        break

print(f"\nGuessed columns: country='{country_col}', year='{year_col}', value='{value_col}'")
if country_col: print("Sample countries:", df[country_col].dropna().unique()[:10])
if year_col: print("Year range:", df[year_col].min(), "-", df[year_col].max())


Columns: ['country_name', 'country_code', 'indicator_name', 'indicator_code', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

Data types:
country_name       object
country_code       object
indicator_name     object
indicator_code     object
2000              float64
2001              float64
2002              float64
2003              float64
2004              float64
2005              float64
2006              float64
2007              float64
2008              float64
2009              float64
2010              float64
2011              float64
2012              float64
2013              float64
2014              float64
2015              float64
2016              float64
2017              float64
2018              float64
2019              float64
2020              float64
2021              float64
2022              float64
dtype: object

Missing 

In [7]:
# 4. Summary stats (numeric columns)
display(df.describe(include='all'))


,country_name,country_code,indicator_name,indicator_code,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
count,266,266,266,266,249.000000,251.000000,253.000000,253.000000,253.000000,257.000000,...,260.000000,260.000000,261.000000,262.000000,259.000000,253.000000,252.000000,249.000000,246.000000,241.000000
unique,266,266,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Aruba,ABW,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,266,266,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,81.219913,81.651725,81.957143,82.349987,82.738922,83.388660,...,86.373186,86.739451,87.162658,87.570448,87.984006,88.264357,88.563480,88.839836,89.128473,89.509081
std,NaN,NaN,NaN,NaN,20.849220,20.507562,20.217341,19.881746,19.548506,19.151960,...,16.720370,16.442111,16.157446,15.879353,15.528850,15.344215,15.117724,14.943717,14.789445,14.603472
min,NaN,NaN,NaN,NaN,18.682303,20.232041,21.771331,23.298164,24.813182,26.316366,...,36.566225,36.438635,36.303098,36.159533,36.007399,35.847086,35.678057,35.500237,35.313097,35.116567
25%,NaN,NaN,NaN,NaN,72.107591,73.708739,73.446224,74.104997,74.735457,75.526964,...,78.620396,79.675693,80.131999,81.140120,81.741906,83.285884,83.598648,84.484982,85.293182,85.912114
50%,NaN,NaN,NaN,NaN,89.306789,89.529782,89.996031,90.021779,90.321598,91.030675,...,93.666058,93.755487,94.247595,94.541203,94.811157,95.168785,95.519036,95.573313,95.791371,96.229362
75%,NaN,NaN,NaN,NaN,98.324423,98.311342,98.340903,98.354428,98.365643,98.541340,...,98.907529,99.026338,99.123134,99.177506,99.235144,99.365136,99.490428,99.544445,99.583189,99.695200
